# Prefix Tuning for Embedding Inversion

In [2]:
import torch
import torch.nn as nn
from modules.data import get_bookcorpus_dataloader
from modules.prenet import PreNet
from modules.encdec import get_encoder, get_gpt2_decoder
from modules.train import train_with_validation

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

resume_from: str = 'saved_models/prenet_prefix_tuning_bookcorpus.pth'

encoder = get_encoder("all-MiniLM-L6-v2")
decoder, tokenizer = get_gpt2_decoder()

prenet = PreNet(
    input_dim=384,
    output_dim=768,
    bottleneck_dim=128,
    prefix_len=20
).to(device)

if resume_from:
    prenet.load_state_dict(torch.load(resume_from, map_location=device))

optimizer = torch.optim.Adam(prenet.parameters(), lr=1e-3)
loss_fct = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

Get dataloaders with cached embeddings.

In [3]:
train_loader, valid_loader = get_bookcorpus_dataloader(
    encoder, tokenizer, max_target_length=64, batch_size=32, embed_batch_size=32, sample=0.03
)

Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 94/94 [00:00<00:00, 161.39it/s]


Train the model.

In [4]:
train_with_validation(prenet, decoder, train_loader, valid_loader, optimizer, loss_fct, 3)

Epoch 1 [Valid]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 94/94 [00:19<00:00,  4.78it/s]


Epoch 1/3 — Train Loss: 2.6788   Val Loss: 2.7082


Epoch 2 [Valid]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 94/94 [00:21<00:00,  4.44it/s]


Epoch 2/3 — Train Loss: 2.6705   Val Loss: 2.7182


Epoch 3 [Valid]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 94/94 [00:19<00:00,  4.78it/s]

Epoch 3/3 — Train Loss: 2.6576   Val Loss: 2.7127


Save the model.

In [12]:
torch.save(prenet.state_dict(), 'saved_models/prenet_prefix_tuning_bookcorpus.pth')

---

## Inference

In [ ]:
def vec_to_text(embedding, decoder, tokenizer, prenet, gen_len=50):
    """
    Given input text, encode it, generate prefix via PreNet, and autoregressively decode output text.
    """
    decoder.eval()
    prenet.eval()
    with torch.no_grad():
        prefix = prenet(embedding.unsqueeze(0))  # (1, prefix_len, model_dim)

        generated = prefix  # initial embeddings
        generated_ids = []
        for _ in range(gen_len):
            outputs = decoder(inputs_embeds=generated)
            next_logits = outputs.logits[:, -1, :]
            next_id = torch.argmax(next_logits, dim=-1).unsqueeze(-1)  # greedy
            generated_ids.append(next_id)
            next_embed = decoder.transformer.wte(next_id)
            generated = torch.cat([generated, next_embed], dim=1)

    gen_ids = torch.cat(generated_ids, dim=1)
    return tokenizer.decode(gen_ids[0].cpu().numpy(), skip_special_tokens=True)

Try inverting a text.

In [11]:
text = "How are you feeling today?."

embedding = encoder.encode(text, convert_to_tensor=True).to(device)

generated_text = vec_to_text(
    embedding, decoder, tokenizer, prenet, 50
)

print(generated_text)

 ... how are you feeling today ? '' '' '' ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
